In [ ]:
import queue
import time
import heapq
from datetime import datetime

#function that estimates the cost to the goal state
def costEstimate(state, goal_state):
  distance = 0
  
  for i in range(3):
    for j in range(3):
      current_value = state[i][j]
      if current_value != -1:
        for row in range(3):
          for col in range(3):
            if goal_state[row][col] == current_value:
              tarX = abs(col - j)
              tarY = row - i
              tarX *= 2

              if tarY < 0: 
                tarY *= 3  
              tarY = abs(tarY)
              
              distance += tarX + tarY     

  return distance

init_state = [[1, 6, 2], [5, 7, 8], [-1, 4, 3]]
goal_state = [[7, 8, 1], [6, -1, 2], [5, 4, 3]]
north, south, east, west = 3, 1, 2, 2

#Priority queue for the frontier and a set implemented as a hash table for the explored states
frontier = queue.PriorityQueue()
explored = set()

#each tuple has f, time, state, g, h, and parent state values
#time is used to serve as a tie breaker in the case where two inputs have the same f value. Order is maintained by time of insertion for FIFO
start = 0, time.time(), init_state, 0, costEstimate(init_state, goal_state), None
frontier.put(start)

#initializing some other needed things 
queue = queue.Queue()
state_tuple = None
while frontier.qsize() > 0:
  state_tuple = frontier.get()
  state = tuple(map(tuple, state_tuple[2]))
  h, t, puzzle, g, f, parent = state_tuple
  tempg = g

  #checking if we have seen this state before, if so go to next iteration, otherwise store it in hash table and continue
  if state in explored:
    continue
  else:
    explored.add(state)
  
  queue.put(state_tuple)

  #checking if current state is the goal state were looking for
  if puzzle == goal_state:
    break

  #Finding the row and col of empty tile
  emptySpaceRow, emptySpaceCol = 0, 0
  for i in range(3):
    for j in range(3):
      if state[i][j] == -1:
        emptySpaceRow, emptySpaceCol = i, j
        break 

  #West
  if emptySpaceCol > 0:
    tmp_state = [list(row) for row in state]
    tmp_state[emptySpaceRow][emptySpaceCol] = tmp_state[emptySpaceRow][emptySpaceCol - 1]
    tmp_state[emptySpaceRow][emptySpaceCol - 1] = -1
    g = g + west
    h = costEstimate(tmp_state, goal_state)
    f = g + h
    tmp_tuple = f, time.time(), tmp_state, g, h, state_tuple
    frontier.put(tmp_tuple)
    f, g, h = 0, tempg, 0

  #North
  if emptySpaceRow > 0:
    tmp_state = [list(row) for row in state]
    tmp_state[emptySpaceRow][emptySpaceCol] = tmp_state[emptySpaceRow - 1][emptySpaceCol]
    tmp_state[emptySpaceRow - 1][emptySpaceCol] = -1
    g = g + south
    h = costEstimate(tmp_state, goal_state)
    f = g + h
    tmp_tuple = f, time.time(), tmp_state, g, h, state_tuple
    frontier.put(tmp_tuple)
    f, g, h = 0, tempg, 0

  #East
  if emptySpaceCol < 2:
    tmp_state = [list(row) for row in state]
    tmp_state[emptySpaceRow][emptySpaceCol] = tmp_state[emptySpaceRow][emptySpaceCol + 1]
    tmp_state[emptySpaceRow][emptySpaceCol + 1] = -1    
    g = g + east
    h = costEstimate(tmp_state, goal_state)
    f = g + h
    tmp_tuple = f, time.time(), tmp_state, g, h, state_tuple
    frontier.put(tmp_tuple)
    f, g, h = 0, tempg, 0

  #South
  if emptySpaceRow < 2:
    tmp_state = [list(row) for row in state]
    tmp_state[emptySpaceRow][emptySpaceCol] = tmp_state[emptySpaceRow + 1][emptySpaceCol]
    tmp_state[emptySpaceRow + 1][emptySpaceCol] = -1
    g = g + north
    h = costEstimate(tmp_state, goal_state)
    f = g + h   
    tmp_tuple = f, time.time(), tmp_state, g, h, state_tuple
    frontier.put(tmp_tuple)
    f, g, h = 0, tempg, 0

#printing out the path and states / expansions
expansions = 0
while queue.qsize() > 0:
  expansions+=1
  curr_state = queue.get()
  if curr_state == None:
    break
  for i in range(3):
    for j in range(3):
      print(str(curr_state[2][i][j]) + ' ', end='')
    print('\n', end='')
  print('________________')
  print('|', curr_state[3], '|',  curr_state[4], '|', '#', expansions, '|')
  print('----------------')


: 